In [32]:
import xml
import xml.etree.ElementTree as ET
import os
from os import path
import glob
from collections import defaultdict, Counter
import numpy as np

from utils import get_ent_info, get_clusters_from_xml

In [33]:
data_dir = "/home/shtoshni/Research/events/data/red/data/source"
source_files = glob.glob("{}/*/*".format(data_dir))

ann_dir = "/home/shtoshni/Research/events/data/red/data/mod_annotation"
ann_files = glob.glob("{}/*/*".format(ann_dir))

output_dir = "/home/shtoshni/Research/events/data/red/error_logs"
if not path.exists(output_dir):
    os.makedirs(output_dir)

## Calculate Types of Coref Chains

In [34]:
parents_type = defaultdict(Counter)
type_properties = defaultdict(Counter)

for source_file in source_files:
    # Read the source doc
    source_lines = open(source_file).readlines()
    source_str = "".join(source_lines)
    
    # Read the annotation file
    base_name = path.basename(source_file)
    dir_name = path.basename(path.dirname(source_file))
    
    ann_file = path.join(path.join(ann_dir, dir_name), base_name + ".RED-Relation.gold.completed.xml")
    
    tree = ET.parse(ann_file)
    root = tree.getroot()
    
    
    for elem in root.iter('relation'):
        parenttype_elem = elem.find('parentsType')
        type_elem = elem.find('type')
        parents_type[parenttype_elem.text][type_elem.text]+= 1
        
        prop_elem = elem.find('properties')
        for tag_elem in prop_elem:
            type_properties[type_elem.text][tag_elem.tag] += 1 
        

for parent_type in parents_type:
    print(parent_type)
    for subtype in parents_type[parent_type]:
        print("\t{}\t{}".format(subtype, parents_type[parent_type][subtype])) 
    print()
print("\n")

for elem_type in type_properties:
    attribute_counts = type_properties[elem_type]
    attribs, attrib_counts = [], []
    for attrib in attribute_counts:
        attribs.append(attrib)
        attrib_counts.append(attribute_counts[attrib])
        
    print(elem_type, *attribs, sep="\t")
    attrib_counts = [''] + attrib_counts
    print(*attrib_counts, sep="\t")
        

TemporalRelations
	TLINK	4209
	ALINK	136

CorefChains
	IDENTICAL	2049
	WHOLE/PART	515
	SET/MEMBER	947
	APPOSITIVE	303
	BRIDGING	383
	REPORTING	596



TLINK	Source	Type	Target	Polarity	ContextualModality	Difficulty
	4209	4209	4209	4209	4209	4209
IDENTICAL	FirstInstance	Coreferring_String
	2049	6512
WHOLE/PART	Whole	Part
	515	815
SET/MEMBER	Set	Member
	947	1614
ALINK	Source	Type	Target	Difficulty
	136	136	136	136
APPOSITIVE	Head	Attribute
	303	303
BRIDGING	Argument	Related_to	FirstInstance	Coreferring_String
	381	433	2	2
REPORTING	Report	Event
	596	1334
